## Configuração Inicial

Configuração e import de bibliotecas.

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from os import getcwd
from tensorflow import keras, reshape
from PIL import Image
import cv2
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

diretorio_treino = 'asl-alphabet/asl_alphabet_train/asl_alphabet_train/'
diretorio_teste = 'asl-alphabet/asl_alphabet_test/'
diretorio_atual = getcwd()

modelo = keras.models.load_model("mymodel.keras")

## Importação dos dados - via diretório

Essa é a forma ideal de importar os dados, já que ela permite a separação das bases de treino e validação no momento da importação.
Porém, precisamos pegar apenas uma parte das bases de treino e validação para fins de velocidade do treino.
Seguindo o downloado e redimensionamento feito na parte de pré-processamento, cada classe terá 150 amostras.

Agora, será montado o dataset de treino, validação e teste, usando keras.

In [21]:
train_dataset = keras.utils.image_dataset_from_directory(
    diretorio_treino,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(200, 200),
    shuffle=True,
    seed=42,
    validation_split=0.25,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)


val_dataset = keras.utils.image_dataset_from_directory(
    diretorio_treino,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(200, 200),
    shuffle=True,
    seed=42,
    validation_split=0.25,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)


test_dataset = keras.utils.image_dataset_from_directory(
    diretorio_teste,
    labels=list(range(0,28)),
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(200, 200),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)

ValueError: The `class_names` passed did not match the names of the subdirectories of the target directory. Expected: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space'], but received: inferred

Tendo as imagens carregadas, vamos aplicar a função de pré-processamento, para assim podermos alimentar o modelo:

In [ ]:
#train_dataset = train_dataset.map(lambda x, y: reajustar_imagem(x, y))
#val_dataset = val_dataset.map(lambda x, y: reajustar_imagem(x, y))
#test_dataset = test_dataset.map(lambda x, y: reajustar_imagem(x, y))

In [6]:
#  Configuração de Pipeline

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

num_classes = 29

pretrained_base = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(200, 200, 3),
    pooling='max',
    classes=num_classes,
    classifier_activation='softmax'
)

pretrained_base.Trainable = False

model = tf.keras.Sequential([
    pretrained_base,
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.3),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

early_stopping = EarlyStopping(
    min_delta=0.00001,
    monitor='val_loss',
    patience=16,
    restore_best_weights=True,
    mode='min'
)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.7,
                              patience=5,
                              min_lr=0.0001)

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.007),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
tf.config.run_functions_eagerly(True)

In [ ]:
epochs = 6

history = model.fit(
  train_dataset,
  validation_data = val_dataset,
  batch_size=32,
  epochs=epochs,
  callbacks=[early_stopping, reduce_lr]
)

In [9]:
predictions = modelo.predict(test_dataset, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = []
for _, labels in test_dataset:
    true_classes.extend(labels.numpy())

confusion_mtx = confusion_matrix(true_classes, predicted_classes)
class_names = [str(i) for i in range(0,29)]
classification_rep = classification_report(true_classes, predicted_classes, target_names=class_names)

print("\nClassification Report:")
print(classification_rep)

1/1 [==============================] - 1s 791ms/step

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         1
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         1
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         1
    

C:\Users\Kafat\anaconda3\envs\tf-directml\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kafat\anaconda3\envs\tf-directml\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kafat\anaconda3\envs\tf-directml\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kaf